In [1]:
from pandas_plink import read_plink
import pandas as pd
import dask 
import dask.array as da
import dask_ml.preprocessing
import numpy as np

In [2]:
#(bim_test, fam_test, G_test) = read_plink('/Users/jenniferblanc/polygenic_adaptation_stratification/output/Simulate_Genotypes/4PopSplit/S2/C1/genos-test_common')
#(bim_gwas, fam_gwas, G_gwas) = read_plink('/Users/jenniferblanc/polygenic_adaptation_stratification/output/Simulate_Genotypes/4PopSplit/S2/C1/genos-gwas_common')

In [2]:
(bim_test, fam_test, G_test) = read_plink('/Users/jenniferblanc/scratch/practice2')
(bim_gwas, fam_gwas, G_gwas) = read_plink('/Users/jenniferblanc/scratch/practice2')


Mapping files: 100%|██████████| 3/3 [00:00<00:00, 39.28it/s]


In [3]:
# Mean Center Test
X = G_test.transpose()
scaler = dask_ml.preprocessing.StandardScaler(with_mean=True, with_std=False).fit(X)
X = scaler.transform(X)

# Mean Center GWAS
M = G_gwas.transpose()
scaler = dask_ml.preprocessing.StandardScaler(with_mean=True, with_std=False).fit(M)
M = scaler.transform(M)

In [4]:
# Calculate Test covariance matrix 
test_cov = dask.array.cov(X)

# SVD of covariance matrix 
u, s, v = dask.array.linalg.svd(test_cov)
n = s.shape[0]

# Make test vector
mat = np.zeros(n)
mat[0:int(n/2)] = 1
mat = mat - mat.mean(axis = 0)
smat = mat ** 2
sums = smat.sum(axis = 0)
stvec = mat / np.sqrt(sums)

# Make pseudoinverse 
vals = dask.array.diag(1/s[0:(n-1)])
pinv = np.dot(u[0:n,0:(n-1)], vals).dot(u[0:n,0:(n-1)].transpose()).compute()

# Calculate Tm
K = np.dot(M, X.transpose()) / (M.shape[1] - 1)
Tm = np.dot(K,pinv).dot(stvec)
Tm = Tm.compute()

print(Tm)

[ 0.09999968  0.1000137   0.09999543  0.09999992  0.10000921  0.09999425
  0.09999653  0.10001465  0.10001292  0.10000441  0.10000795  0.10001284
  0.10001205  0.09999748  0.10000732  0.09999212  0.10002119  0.10000662
  0.10000992  0.09998401  0.09997464  0.10001134  0.10001307  0.09999827
  0.10002473  0.09999409  0.09999488  0.10000646  0.09999984  0.09998259
  0.09999827  0.09999732  0.09998582  0.10000693  0.09998338  0.10001433
  0.10000756  0.10000283  0.10000638  0.10000795  0.10000031  0.10000725
  0.09998944  0.1000026   0.09999157  0.10001118  0.09999787  0.10000094
  0.09998913  0.100013   -0.10000418 -0.10000087 -0.09999976 -0.10000441
 -0.10000827 -0.10000496 -0.09999031 -0.10000425 -0.10000339 -0.1000089
 -0.10001591 -0.0999978  -0.10000701 -0.10000473 -0.10000835 -0.10000906
 -0.09999764 -0.1000015  -0.09999913 -0.10001071 -0.10002229 -0.10000504
 -0.10000733 -0.1000256  -0.10000536 -0.09999047 -0.10000922 -0.09997708
 -0.09999315 -0.10001056 -0.10000047 -0.09999661 -0.

In [5]:
# Calculate Test covariance matrix 
test_cov = dask.array.cov(X)

# SVD of covariance matrix 
u, s, v = dask.array.linalg.svd_compressed(test_cov, test_cov.shape[0])
n = s.shape[0]

# Make test vector
mat = np.zeros(n)
mat[0:int(n/2)] = 1
mat = mat - mat.mean(axis = 0)
smat = mat ** 2
sums = smat.sum(axis = 0)
stvec = mat / np.sqrt(sums)

# Make pseudoinverse 
vals = dask.array.diag(1/s[0:(n-1)])
pinv = np.dot(u[0:n,0:(n-1)], vals).dot(u[0:n,0:(n-1)].transpose())

# Calculate Tm
K = np.dot(M, X.transpose()) / (M.shape[1] - 1)
Tm = np.dot(K,pinv).dot(stvec)

Tm.compute()

array([ 0.09999968,  0.1000137 ,  0.09999543,  0.09999992,  0.10000921,
        0.09999425,  0.09999653,  0.10001465,  0.10001292,  0.10000441,
        0.10000795,  0.10001284,  0.10001205,  0.09999748,  0.10000732,
        0.09999212,  0.10002119,  0.10000662,  0.10000992,  0.09998401,
        0.09997464,  0.10001134,  0.10001307,  0.09999827,  0.10002473,
        0.09999409,  0.09999488,  0.10000646,  0.09999984,  0.09998259,
        0.09999827,  0.09999732,  0.09998582,  0.10000693,  0.09998338,
        0.10001433,  0.10000756,  0.10000283,  0.10000638,  0.10000795,
        0.10000031,  0.10000725,  0.09998944,  0.1000026 ,  0.09999157,
        0.10001118,  0.09999787,  0.10000094,  0.09998913,  0.100013  ,
       -0.10000418, -0.10000087, -0.09999976, -0.10000441, -0.10000827,
       -0.10000496, -0.09999031, -0.10000425, -0.10000339, -0.1000089 ,
       -0.10001591, -0.0999978 , -0.10000701, -0.10000473, -0.10000835,
       -0.10000906, -0.09999764, -0.1000015 , -0.09999913, -0.10

In [7]:
# SVD of covariance matrix 
u, s, v = dask.array.linalg.svd_compressed(X, X.shape[0])
n = s.shape[0]

# Make test vector
mat = np.zeros(n)
mat[0:int(n/2)] = 1
mat = mat - mat.mean(axis = 0)
smat = mat ** 2
sums = smat.sum(axis = 0)
stvec = mat / np.sqrt(sums)

# Make pseudoinverse 
s = da.square(s) / (X.shape[1] -1)
#print(s.compute())
vals = dask.array.diag(1/(s[0:(n-1)]))
pinv = np.dot(u[0:n,0:(n-1)], vals).dot(u[0:n,0:(n-1)].transpose())

# Calculate Tm
K = np.dot(M, X.transpose()) / (M.shape[1] - 1)
Tm = np.dot(K,pinv).dot(stvec)

Tm.compute()

array([ 0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,
        0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,
        0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,
        0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,
        0.1,  0.1,  0.1,  0.1,  0.1,  0.1, -0.1, -0.1, -0.1, -0.1, -0.1,
       -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1,
       -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1,
       -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1,
       -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1,
       -0.1])

In [24]:
# SVD of covariance matrix 
u, s = dask.array.linalg.eig(X, X.shape[0])
n = s.shape[0]

# Make test vector
mat = np.zeros(n)
mat[0:int(n/2)] = 1
mat = mat - mat.mean(axis = 0)
smat = mat ** 2
sums = smat.sum(axis = 0)
stvec = mat / np.sqrt(sums)

# Make pseudoinverse 
s = da.square(s)
print(s.compute())
vals = dask.array.diag(1/(s[0:(n-1)]))
pinv = np.dot(u[0:n,0:(n-1)], vals).dot(u[0:n,0:(n-1)].transpose())

# Calculate Tm
K = np.dot(M, X.transpose()) / (M.shape[1] - 1)
Tm = np.dot(K,pinv).dot(stvec)

Tm.compute()

AttributeError: module 'dask.array.linalg' has no attribute 'eig'

In [22]:
#file_name="/Users/jenniferblanc/Test.txt"
#outF = open(file_name, "w")
#for i in range(Tm.shape[0]):
#    print(i)
#    outF.write(str(Tm[i].compute()))
#outF.close()

print(X)

dask.array<sub, shape=(100, 45361), dtype=float64, chunksize=(100, 1024), chunktype=numpy.ndarray>


In [9]:
Tvec = np.loadtxt("/Users/jenniferblanc/polygenic_adaptation_stratification/output/Calculate_Tm/4PopSplit/S2/Tvec.txt")

In [18]:
#da.store(Tm, /Users/jenniferblanc/scratch/testTm")
da.to_hdf5('/Users/jenniferblanc/scratch/testTm.hdf5', '/output', Tm)